In [1]:
from sklearn.ensemble import RandomForestRegressor
import joblib
import numpy as np
from sklearn.metrics import accuracy_score
from quadratic_weighted_kappa import quadratic_weighted_kappa
import pandas as pd

In [3]:
x = joblib.load('essay_ease10_sbert768_simbow_langerr_780_normalized_asap3')
asap1 = joblib.load("asap3_new_adjudication")
x.shape

(1726, 780)

In [4]:
asap1.head()

,essay,rater1_domain1,rater2_domain1,domain1_score,higher,mean,sum,check
0,The features of the setting affect the cyclist...,1,1,1,1,1.0,2,True
1,The features of the setting affected the cycli...,2,2,2,2,2.0,4,True
2,Everyone travels to unfamiliar places. Sometim...,1,1,1,1,1.0,2,True
3,I believe the features of the cyclist affected...,1,1,1,1,1.0,2,True
4,The setting effects the cyclist because of the...,2,2,2,2,2.0,4,True


In [24]:
y = asap1["sum"]

In [25]:
def get_feature_names_extended():
    ease_feats = ['Answer Length', 'Word Counts', 'Average Word Length', 'Good n-gram', 'Prompt Overlap',
              'Prompt Overlap (synonyms)', 'Punctuation Counts', 'Spelling Error', 'Unique Words', 'Prompt Similarity SBert']

    sbert_feats = []
    sbert_dim = 768

    for i in range(0, sbert_dim):
    	fname = "sbert_" + str(i) 
    	sbert_feats.append(fname)
    
    prompt_similarity_bow = ["Prompt Similarity BOW"]
    lang_error = ["Language Error"]
    
    feature_names = ease_feats + prompt_similarity_bow + lang_error + sbert_feats 

    print("len feature names: ", len(feature_names))
    
    return feature_names

feature_names = get_feature_names_extended()[:12]

len feature names:  780


In [26]:
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

kf = KFold(n_splits=5, shuffle=True, random_state=42)

print(kf)

KFold(n_splits=5, random_state=42, shuffle=True)


In [27]:
model2 = RandomForestRegressor(n_estimators = 100)

In [28]:
qwk_scores = []

test_indices = []
pred_labels = []

for train_index, test_index in kf.split(x, y):
#     print("train_index: ", len(train_index))
#     print("test_index: ", len(test_index))
#     print("train_index: ", train_index[:20])
#     print("test_index: ", test_index[:20])
    
    X_train, X_test, Y_train, Y_test = x[train_index], x[test_index], y[train_index], y[test_index]

    #d_train = xgboost.DMatrix(X_train, label=Y_train, feature_names=feature_names)
    #d_test = xgboost.DMatrix(X_test, label=Y_test, feature_names=feature_names)
         
    model2.fit(X_train, Y_train)
    
    #rf.fit(X_train, Y_train)
    
    #predict = model.predict(d_test)
    predict = model2.predict(X_test)
    #predict = rf.predict(X_test)
    predict = np.round(predict)
    
    pred_labels.extend(predict)
    test_indices.extend(test_index)

    result = quadratic_weighted_kappa(Y_test, predict)
    
    print("Qwk : ", result)
    
    qwk_scores.append(result)
    
    
np.mean(qwk_scores)

Qwk :  0.7230005604034906
Qwk :  0.6391811202729598
Qwk :  0.6749112639817205
Qwk :  0.7122558134196451
Qwk :  0.7076910699515018


0.6914079656058636

In [33]:
print(len(pred_labels))
print(len(test_indices))

1726
1726


In [34]:
pred_labels_int = list(map(int, pred_labels))

In [35]:
new_score = np.zeros(len(pred_labels))
new_score

array([0., 0., 0., ..., 0., 0., 0.])

In [36]:
new_score[test_indices] = pred_labels_int

In [37]:
joblib.dump(new_score, 'score_model_normalized_asap3_rf_sum')

['score_model_normalized_asap3_rf_sum']